In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data_v1_max_72_h.csv')

In [2]:
df2 = df.copy()

In [3]:

# df = your hourly table
df["visit_start_datetime"] = pd.to_datetime(df["visit_start_datetime"])
df["hour_ts"] = df["visit_start_datetime"] + pd.to_timedelta(df["measure_time"], unit="h")

In [4]:
H_DAYS = 30
H_HOURS = 24 * H_DAYS

df["Y_30d"] = ((df["death_hours"].notna()) & (df["death_hours"] <= H_HOURS)).astype(int)

In [5]:
MAX_HOURS = 48
df = df[(df["measure_time"] >= 0) & (df["measure_time"] < MAX_HOURS)].copy()

# remove hours beyond LOS
df = df[df["measure_time"] <= df["length_of_stay_hours"]].copy()

# remove hours after death
df = df[(df["death_hours"].isna()) | (df["measure_time"] < df["death_hours"])].copy()

In [33]:
df['peep_mean'].ge(15)

1          False
2          False
7          False
9          False
11         False
           ...  
1721173    False
1721178    False
1721182    False
1721183    False
1721186    False
Name: peep_mean, Length: 515545, dtype: bool

In [37]:
# a crude "mechanical ventilation present" proxy
df["mv_proxy"] = (
    df["peep_mean"].fillna(0).ge(5) &
    df["peak_mean"].fillna(0).ge(15) 
).astype(int)

# first hour of mv_proxy per visit
df = df.sort_values(["visit_occurrence_id", "measure_time"])
first_mv = (
    df[df["mv_proxy"] == 1]
    .groupby("visit_occurrence_id")["measure_time"]
    .min()
    .rename("t_mv_start")
)
df = df.merge(first_mv, on="visit_occurrence_id", how="left")

In [38]:
# A_t = 1 if mv_proxy starts in (t, t+1]
df["A_intub_within_1h"] = (
    df["t_mv_start"].notna() &
    (df["t_mv_start"] > df["measure_time"]) &
    (df["t_mv_start"] <= df["measure_time"] + 1)
).astype(int)

In [40]:
df["already_mv"] = df["t_mv_start"].notna() & (df["measure_time"] >= df["t_mv_start"])
df["eligible"] = ~df["already_mv"]
df = df[df["eligible"]].copy()

In [41]:
np.random.seed(42)  # reproducibility

# e.g., 30% prevalence
AHRF_PREVALENCE = 0.30
df["ahrf_proxy"] = np.random.binomial(1, AHRF_PREVALENCE, size=len(df))

df = df[df["ahrf_proxy"] == 1].copy()

In [42]:
FEATURES = [
    "measure_time",
    "age",
    "fio2_mean",
    "map_mean", "sbp_mean", "dbp_mean",
    "temp_mean",
    "wbc_mean", "hemoglobin_mean", "platelets_mean",
    "sodium_mean", "potassium_mean", "chloride_mean", "glucose_mean",
    "bun_mean", "creatinine_mean",
    "crp_mean",
    # include vent mechanics as confounders even if also used in action proxy; sensitivity needed
    "peep_mean", "peak_mean",
]

In [43]:
from sklearn.model_selection import train_test_split

enc_ids = df["visit_occurrence_id"].unique()
train_ids, test_ids = train_test_split(enc_ids, test_size=0.2, random_state=42)

df_tr = df[df["visit_occurrence_id"].isin(train_ids)].copy()
df_te = df[df["visit_occurrence_id"].isin(test_ids)].copy()

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

prop_model = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)),
    ("clf", LogisticRegression(max_iter=300))
])

prop_model.fit(df_tr[FEATURES], df_tr["A_intub_within_1h"])
p_tr = prop_model.predict_proba(df_tr[FEATURES])[:, 1].clip(0.01, 0.99)
p_te = prop_model.predict_proba(df_te[FEATURES])[:, 1].clip(0.01, 0.99)

p_marg = df_tr["A_intub_within_1h"].mean()

def stabilized_iptw(a, p, p_marg):
    num = np.where(a == 1, p_marg, 1 - p_marg)
    den = np.where(a == 1, p, 1 - p)
    w = num / den
    return np.clip(w, 0, np.quantile(w, 0.99))

df_tr["w"] = stabilized_iptw(df_tr["A_intub_within_1h"].values, p_tr, p_marg)
df_te["w"] = stabilized_iptw(df_te["A_intub_within_1h"].values, p_te, p_marg)

OUTCOME_FEATURES = FEATURES + ["A_intub_within_1h"]

outcome_model = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)),
    ("clf", LogisticRegression(max_iter=500))
])

outcome_model.fit(
    df_tr[OUTCOME_FEATURES],
    df_tr["Y_30d"],
    clf__sample_weight=df_tr["w"]
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['fio2_mean' 'bun_mean']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['fio2_mean' 'bun_mean']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['fio2_mean' 'bun_mean']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Sk

Pipeline(steps=[('impute', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler(with_mean=False)),
                ('clf', LogisticRegression(max_iter=500))])

In [45]:
def predict_counterfactuals(df, model, features):
    X = df[features].copy()

    X1 = X.copy()
    X1["A_intub_within_1h"] = 1
    r1 = model.predict_proba(X1[features + ["A_intub_within_1h"]])[:, 1]

    X0 = X.copy()
    X0["A_intub_within_1h"] = 0
    r0 = model.predict_proba(X0[features + ["A_intub_within_1h"]])[:, 1]

    ard = r0 - r1
    return r1, r0, ard

r_intub, r_wait, ard = predict_counterfactuals(df_te, outcome_model, FEATURES)

df_te["risk_intub_now_30d"] = r_intub
df_te["risk_wait_1h_30d"] = r_wait
df_te["ard_wait_minus_intub"] = ard

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['fio2_mean' 'bun_mean']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['fio2_mean' 'bun_mean']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [46]:
from sklearn.metrics import roc_auc_score, brier_score_loss

pred_obs = outcome_model.predict_proba(df_te[OUTCOME_FEATURES])[:, 1]
auc = roc_auc_score(df_te["Y_30d"], pred_obs)
brier = brier_score_loss(df_te["Y_30d"], pred_obs)
print("AUC:", auc, "Brier:", brier)

AUC: 0.6504323737545661 Brier: 0.10678558980505787


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['fio2_mean' 'bun_mean']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [48]:
import numpy as np
import pandas as pd

def _predict_cf_risks_row(row: pd.Series, model, features):
    """
    Returns (risk_intub_now, risk_wait_1h, ard_wait_minus_intub)
    computed by plugging A=1 vs A=0 into the trained outcome model.
    """
    x = row[features].to_frame().T.copy()

    x1 = x.copy()
    x1["A_intub_within_1h"] = 1
    r1 = float(model.predict_proba(x1[features + ["A_intub_within_1h"]])[:, 1][0])

    x0 = x.copy()
    x0["A_intub_within_1h"] = 0
    r0 = float(model.predict_proba(x0[features + ["A_intub_within_1h"]])[:, 1][0])

    ard = r0 - r1
    return r1, r0, ard

In [49]:
from sklearn.base import clone

def bootstrap_ci_for_row(
    row: pd.Series,
    df_train: pd.DataFrame,
    outcome_model,
    features,
    outcome_features,
    visit_col="visit_occurrence_id",
    y_col="Y_30d",
    weight_col="w",
    n_boot=300,
    random_state=42
):
    rng = np.random.default_rng(random_state)
    visits = df_train[visit_col].unique()

    r1_list, r0_list, ard_list = [], [], []

    for _ in range(n_boot):
        boot_visits = rng.choice(visits, size=len(visits), replace=True)
        boot_df = df_train[df_train[visit_col].isin(boot_visits)].copy()

        # Refit outcome model on the bootstrap sample
        m = clone(outcome_model)
        if weight_col in boot_df.columns:
            m.fit(boot_df[outcome_features], boot_df[y_col], clf__sample_weight=boot_df[weight_col])
        else:
            m.fit(boot_df[outcome_features], boot_df[y_col])

        r1, r0, ard = _predict_cf_risks_row(row, m, features)
        r1_list.append(r1); r0_list.append(r0); ard_list.append(ard)

    r1_arr = np.array(r1_list)
    r0_arr = np.array(r0_list)
    ard_arr = np.array(ard_list)

    ci = lambda a: (float(np.quantile(a, 0.025)), float(np.quantile(a, 0.975)))

    return {
        "risk_intub_now_ci95": ci(r1_arr),
        "risk_wait_1h_ci95": ci(r0_arr),
        "ard_ci95": ci(ard_arr),
        "p_benefit": float(np.mean(ard_arr > 0.0)),  # P(wait increases mortality) i.e., benefit of intubating now
        "boot_draws": len(ard_arr)
    }

In [50]:
def local_drivers_for_ard(row, model, features, step_frac=0.05):
    """
    Returns ranked drivers based on how much ARD changes when each feature is perturbed.
    step_frac: relative step size (5% of feature value) with fallback to 1 unit if near zero.
    """
    r1_base, r0_base, ard_base = _predict_cf_risks_row(row, model, features)
    drivers = []

    for f in features:
        if f == "measure_time":
            continue  # usually not a physiologic "driver" to display

        v = row[f]
        if pd.isna(v):
            continue

        # Step size
        step = step_frac * abs(v)
        if step < 1e-6:
            step = 1.0

        row_up = row.copy()
        row_up[f] = v + step

        _, _, ard_up = _predict_cf_risks_row(row_up, model, features)
        delta = ard_up - ard_base  # positive means increasing feature increases ARD (harm of waiting)

        drivers.append((f, float(delta)))

    # rank by absolute impact on ARD
    drivers = sorted(drivers, key=lambda x: abs(x[1]), reverse=True)[:5]
    return drivers

In [51]:
def decision_support_output(
    row: pd.Series,
    outcome_model,
    features,
    df_train_for_bootstrap: pd.DataFrame,
    outcome_features,
    tau=0.02,                 # threshold for recommending intubation (2% absolute)
    n_boot=300
):
    r_intub, r_wait, ard = _predict_cf_risks_row(row, outcome_model, features)

    boot = bootstrap_ci_for_row(
        row=row,
        df_train=df_train_for_bootstrap,
        outcome_model=outcome_model,
        features=features,
        outcome_features=outcome_features,
        n_boot=n_boot
    )

    # Recommendation rule (can be extended with gates: DNI, etc.)
    recommend = "Intubate now" if ard >= tau and boot["p_benefit"] >= 0.80 else "Continue noninvasive and reassess"

    drivers = local_drivers_for_ard(row, outcome_model, features)

    # Pretty formatting
    def pct(x): return f"{100*x:.1f}%"
    def pct_ci(ci): return f"{pct(ci[0])} to {pct(ci[1])}"

    return {
        "visit_occurrence_id": row.get("visit_occurrence_id"),
        "measure_time_hour": int(row.get("measure_time")),
        "If intubate now: predicted 30-day mortality": pct(r_intub),
        "If wait 1 hour: predicted 30-day mortality": pct(r_wait),
        "Difference (wait − intubate)": f"+{pct(ard)} absolute" if ard >= 0 else f"{pct(ard)} absolute",
        "ARD 95% CI": pct_ci(boot["ard_ci95"]),
        "P(benefit)": f"{boot['p_benefit']:.2f}",
        "Recommendation": recommend,
        "Key drivers (local ΔARD ranking)": drivers,
    }

In [54]:
row

visit_occurrence_id            2
measure_time                   1
person_id                  55044
gender                       NaN
year_of_birth               1973
                          ...   
t_mv_start                   NaN
w                       1.009911
risk_intub_now_30d      0.761505
risk_wait_1h_30d        0.051394
ard_wait_minus_intub   -0.710111
Name: 17, Length: 96, dtype: object

In [ ]:
//*
{
  'visit_occurrence_id': 84217391,
  'measure_time_hour': 6,

  'If intubate now: predicted 30-day mortality': '18.7%',
  'If wait 1 hour: predicted 30-day mortality': '23.9%',

  'Difference (wait − intubate)': '+5.2% absolute',
  'ARD 95% CI': '1.6% to 8.9%',
  'P(benefit)': '0.91',

  'Recommendation': 'Intubate now',

  'Key drivers (local ΔARD ranking)': [
      ('fio2_mean', +0.031),
      ('peep_mean', +0.018),
      ('map_mean', -0.014),
      ('crp_mean', +0.011),
      ('creatinine_mean', +0.009)
  ]
}
    *//  

How a clinician would read this

At ICU hour 6, given the patient’s current physiology:

• If intubated now, predicted 30-day mortality is 18.7%
• If intubation is deferred for 1 hour, predicted mortality rises to 23.9%
• Absolute harm of waiting: +5.2% (95% CI 1.6–8.9)
• Probability that waiting is worse: 91%

✅ Recommendation: Intubate now

🔍 Primary contributors to harm of waiting:
	•	Rising FiO₂ requirement
	•	Elevated PEEP
	•	Worsening systemic inflammation (CRP)
	•	Early renal dysfunction
	•	MAP partially compensatory but insufficient



In [53]:
OUTCOME_FEATURES = FEATURES + ["A_intub_within_1h"]

first_visit = df_te["visit_occurrence_id"].iloc[0]
sub = df_te[df_te["visit_occurrence_id"] == first_visit].sort_values("measure_time")

row = sub.iloc[0]   # first available hour for that visit

out = decision_support_output(
    row=row,
    outcome_model=outcome_model,
    features=FEATURES,
    df_train_for_bootstrap=df_tr,
    outcome_features=OUTCOME_FEATURES,
    tau=0.02,
    n_boot=300
)

out

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['fio2_mean' 'bun_mean']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['fio2_mean' 'bun_mean']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['fio2_mean' 'bun_mean']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Sk

{'visit_occurrence_id': 2,
 'measure_time_hour': 1,
 'If intubate now: predicted 30-day mortality': '76.2%',
 'If wait 1 hour: predicted 30-day mortality': '5.1%',
 'Difference (wait − intubate)': '-71.0% absolute',
 'ARD 95% CI': '-95.1% to 5.7%',
 'P(benefit)': '0.47',
 'Recommendation': 'Continue noninvasive and reassess',
 'Key drivers (local ΔARD ranking)': [('age', -0.004002553535231934),
  ('sbp_mean', 0.003694561203093638),
  ('platelets_mean', 0.0027261149670494422),
  ('potassium_mean', -0.0026323000163914756),
  ('dbp_mean', -0.0014190750307538913)]}